In [1]:
import os  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
import math
import csv


In [ ]:
#PARENT_FOlDER is where to access csv
#SAVE_FILE is where to save the RP IMAGES 

PARENT_FOLDER = "/glucose-prediction/Dataset/CSV"
SAVE_FILE = "/glucose-prediction/Dataset/CSV"



In [ ]:
START_PARTICIPANT = 1  
END_PARTICIPANT = 14  

for participant_number in range(START_PARTICIPANT, END_PARTICIPANT+1):
    participant_folder = f"P{str(participant_number)}" 
    participant_path = os.path.join(PARENT_FOLDER, participant_folder)
    
    if os.path.isdir(participant_path):
        participant_number = int(participant_folder[1:])

        # Read the CSV files for the participant
        df_temp = pd.read_csv(os.path.join(participant_path, f"TEMP_{str(participant_number).zfill(3)}.csv"))
        df_bvp = pd.read_csv(os.path.join(participant_path, f"BVP_{str(participant_number).zfill(3)}.csv"))
        df_eda = pd.read_csv(os.path.join(participant_path, f"EDA_{str(participant_number).zfill(3)}.csv"))

        min_length = min(len(df_temp), len(df_bvp), len(df_eda))
        df_temp = df_temp.head(min_length)
        df_bvp = df_bvp.head(min_length)
        df_eda = df_eda.head(min_length)

        # Convert the 'datetime' column to datetime data type and set it as the index
        df_temp['datetime'] = pd.to_datetime(df_temp['datetime'])
        df_temp.set_index('datetime', inplace=True)
        df_bvp['datetime'] = pd.to_datetime(df_bvp['datetime'])
        df_bvp.set_index('datetime', inplace=True)
        df_eda['datetime'] = pd.to_datetime(df_eda['datetime'])
        df_eda.set_index('datetime', inplace=True)

        """Hertz (Hz) is the unit of frequency defined as one cycle per second. 
            Data sampled at 4 Hz, it means 4 samples per second.
        To resample data at 4 Hz, need the time equivalent of 4 Hz, 
        which is 0.25 second (since 1 Hz = 1 cycle per second). (1 sample per 0.25 sec)
        0.25 second becomes 250 milliseconds.
        So to resample your data at 4 Hz,  use '250L' 
        which represents 250 milliseconds:"""
        df_bvp_resampled = df_bvp.resample('250L').mean()

        # Concatenate the resampled DataFrames along the columns axis to create the merged DataFrame
        merged_df = pd.concat([df_temp, df_bvp_resampled, df_eda], axis=1)


        cleaned_save_file_path = os.path.join(participant_path, f"COMBO_{str(participant_number).zfill(3)}.csv")
        merged_df.to_csv(cleaned_save_file_path)


In [ ]:
for participant_number in range(START_PARTICIPANT, END_PARTICIPANT+1):
    participant_folder = f"P{str(participant_number)}" 
    participant_path = os.path.join(PARENT_FOLDER, participant_folder)
    
    if os.path.isdir(participant_path):
        combo_file_path = os.path.join(participant_path, f"COMBO_{str(participant_number).zfill(3)}.csv")

        df = pd.read_csv(combo_file_path)

        df.dropna(inplace=True)

        df.to_csv(combo_file_path, index=False)
